In [57]:
import os, requests, pandas as pd, geopandas as gpd
import re
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pyproj import Transformer

In [58]:
# Set to False while testing code, should be True for updates
overwrite_files = False

# Set to True to geocode all locations (including those that have already been geocoded) from scratch
# Only used to ensure consistency of coordinate sources if geocoding method changes
geocode_all = False # Should stay False for efficiency

# Set to False to re-attempt geocoding for previously unmatched locations from the last 180 days
exclude_past_no_match = False

### Load recent data (last 180 days)

In [59]:
# Note that limit defaults to 1000
data_url='https://data.providenceri.gov/resource/rz3y-pz8v.json?%24limit=20000'
response=requests.get(data_url)
results = response.json()
results_df = pd.DataFrame.from_records(results)

print(len(results_df), "rows")
results_df.head()

8671 rows


,casenumber,location,reported_date,month,year,offense_desc,statute_code,statute_desc,counts,reporting_officer
0,2024-00002108,200 Block HANOVER ST,2024-01-09T11:23:00.000,1,2024,Auto Towed,Not Used,Parking or MV Violation,0,EEspinal
1,2024-00002106,100 Block BROAD ST,2024-01-09T11:13:00.000,1,2024,"Assault, Simple",11-5-3,SIMPLE ASSAULT OR BATTERY,1,Central Station
2,2024-00002103,400 Block DOUGLAS AVE,2024-01-09T11:03:00.000,1,2024,Motor Vehicle Theft,11-41-1,LARCENY/U $1500 - AUTO THEFT,1,Central Station
3,2024-00002089,100 Block JULIAN ST,2024-01-09T09:42:00.000,1,2024,Motor Vehicle Theft,11-41-1,LARCENY/O $1500 - AUTO THEFT,1,DRosso
4,2024-00002066,700 Block WESTMINSTER ST,2024-01-09T07:07:00.000,1,2024,Trespass,Not Used,No Violations,0,MCamardo


### Load previously geocoded data

In [60]:
# Successfully geocoded records
past_df = pd.read_csv(os.path.join('..', 'outputs', 'all', 'geocoded_results.csv'))
# Records that could not be geocoded
past_no_matches = pd.read_csv(os.path.join('..', 'outputs', 'all', 'no_location_matches.csv'))

### Isolate new cases

In [61]:
if geocode_all:
    # concatenate past_df, results_df, and past_no_matches
    df_all = pd.concat((past_df, results_df, past_no_matches))
    # drop dupilcate records
    df_all = df_all.drop_duplicates(subset=['casenumber', 'reported_date', 'offense_desc', 'statute_code', 'statute_desc'])
    results_df = df_all
else:
    df2 = results_df.copy()
    # Identify new records by case number and time
    df1 = past_df[['casenumber', 'reported_date']]

    if exclude_past_no_match:
        # Exclude past locations which returned no location matches
        df1 = pd.concat((df1, past_no_matches[['casenumber', 'reported_date']]))
    else:
        # Add previously unmatched locations to the df to attempt geocoding again
        df2 = pd.concat((df2, past_no_matches))
        df2 = df2.drop_duplicates(subset=['casenumber', 'reported_date', 'offense_desc', 'statute_code', 'statute_desc'])
    merged = df2.merge(df1, on=['casenumber', 'reported_date'], how='left', indicator=True)

    # Identify new case records
    new_records = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])
    # results_df is the DataFrame which will be geocoded
    results_df = new_records

results_df.head()

,casenumber,location,reported_date,month,year,offense_desc,statute_code,statute_desc,counts,reporting_officer,unique_id,violent_cat,property_cat
30,2024-00001955,FINANCE WAY & MEMORIAL BLVD,2024-01-08T17:09:00.000,1,2024,Request for Assistance,Not Used,No Violations,0,DAnderson,NaN,NaN,NaN
33,2024-00500049,S CRIMEA,2024-01-08T15:06:59.000,1,2024,Larceny from Motor Vehicle,11-41-1,LARCENY/U $1500 - FROM MV,1,NField,NaN,NaN,NaN
34,2024-00001924,F C GREENE MEMORIAL BLVD,2024-01-08T14:57:00.000,1,2024,"Larceny, Other",11-41-7,LARCENY FROM THE PERSON - ALL OTH LARCENY,1,Central Station,NaN,NaN,NaN
42,2024-00001868,BROAD ST,2024-01-08T09:31:00.000,1,2024,"Larceny, Other",11-41-1,LARCENY/U $1500 - ALL OTH LARCENY,1,Central Station,NaN,NaN,NaN
46,2024-00001837,600 Block PLEASANT VALLEY PKY,2024-01-08T07:23:00.000,1,2024,Property Damage,Not Used,No violations,0,KBass,NaN,NaN,NaN


### Load E911 data
(needed to geocode block locations)

In [62]:
FILEPATH = os.path.join('.', 'inputs', 'e911', 'FACILITY_Sites_E911.shp')

# gdf = GeoDataFrame of E911 sites
gdf = gpd.read_file(FILEPATH)
gdf['Latitude'] = gdf.geometry.y
gdf['Longitude'] = gdf.geometry.x

# Limit E911 sites to Providence
gdf = gdf[gdf['MSAGComm'] == 'PROVIDENCE']

# Standardize street names
gdf['St_Full'] = gdf['St_Full'].str.lower().str.strip()
gdf['St_Name'] = gdf['St_Name'].str.lower().str.strip()

print("All columns:")
print(gdf.columns)

# View columns of interest
to_view = ['Add_Full', 'Add_Number', 'St_Full', 'St_Name', 'St_Alias1', 'St_Alias2', 'St_Alias3', 'St_Alias4', 'St_Alias5', 'Comments', 'Latitude', 'Longitude']
gdf[to_view].head()

All columns:
Index(['OBJECTID', 'DateUpdate', 'Site_GUID', 'SiteType', 'Add_Full',
       'AddNumFull', 'AddNum_Pre', 'Add_Number', 'AddNum_Suf', 'St_Full',
       'St_PreMod', 'St_PreDir', 'St_PreTyp', 'St_Name', 'St_PosType',
       'St_PosDir', 'St_PosMod', 'MSAGComm', 'ESN', 'State', 'Post_Code',
       'Country', 'St_Alias1', 'St_Alias2', 'St_Alias3', 'St_Alias4',
       'St_Alias5', 'Comments', 'geometry', 'Latitude', 'Longitude'],
      dtype='object')


,Add_Full,Add_Number,St_Full,St_Name,St_Alias1,St_Alias2,St_Alias3,St_Alias4,St_Alias5,Comments,Latitude,Longitude
31510,121 FARMINGTON AV,121.0,farmington av,farmington,None,None,None,None,None,2 stry lght blu wht trm frnt prch blk rail,41.805352,-71.459654
31661,12 CROWN ST,12.0,crown st,crown,None,None,None,None,None,wht 3 decka cncrt stps blk rail blcny,41.805878,-71.454730
38637,2 DEERFIELD TERR,2.0,deerfield terr,deerfield,None,None,None,None,None,brwn shngles 2stry wht trim,41.785447,-71.420707
38638,3 DEERFIELD TERR,3.0,deerfield terr,deerfield,None,None,None,None,None,2stry wht split blk shttrs frnt prch,41.785194,-71.420797
38652,98 CYR ST,98.0,cyr st,cyr,None,None,None,None,None,"tan brck 1stry wht trim brwn shttrs R grg, see VC",41.785208,-71.404273


### Geocode the new data

In [63]:
def categorize_address(address):
    """Categorizes locations by type and extracts necessary compononents for geocoding

    Parameters
    ----------
    address : str
        a singular value from the 'location' column of the case logs DataFrame

    Returns
    ----------
    tuple[int, list[str]]
        1. an integer value indicating the address category
              0 = Block
              1 = Intersection
              2 = Landmark
        2. Address components needed to geocode the location
              [block_number, street_name] for category 0
              [street_name1, street_name2] for category 1
              address (the input parameter) for category 2
    """
    # Handles rare case where location == nan
    if not isinstance(address, str):
       return (None, [])

    # Define block locations as having a number followed by 'Block', followed by any combination
    # of alphanumeric characters and spaces
    block_pattern = r'(\d+)\s+Block\s+([\dA-Za-z\s\']+)'

    # Define intersection locations as having any combination of alphanumeric characters and spaces
    # separated by 'AND', '&', or 'CORNER OF'
    address = address.replace(' AND ', ' & ').replace('CORNER OF ', '')
    intersection_pattern = r'([\dA-Za-z\s]+)\s*&\s*([\dA-Za-z\s]+)'

    # Check if the address matches the block format
    block_match = re.match(block_pattern, address)
    if block_match:
        block_number = block_match.group(1)
        street_name = block_match.group(2).strip().lower()
        return (0, (block_number, street_name))

    # Check if the address matches the intersection format
    intersection_match = re.match(intersection_pattern, address)
    if intersection_match:
        street_name1 = intersection_match.group(1).strip().lower()
        street_name2 = intersection_match.group(2).strip().lower()
        return (1, (street_name1, street_name2))

    # If the address does not match either format, treat it as a landmark
    return (2, address)

## Boundary filtering

In [64]:
def in_providence(latitude, longitude):
    # Load the GeoDataFrame containing the polygon from the GeoPackage file
    gdf_polygon = gpd.read_file(os.path.join('.', 'inputs', 'pvd_boundary.gpkg'), layer='pvd_boundary')
    buffer_distance = 0.001
    gdf_polygon = gdf_polygon['geometry'].iloc[0].buffer(buffer_distance)

    # Create a Point geometry from the given latitude and longitude
    point = Point(longitude, latitude)

    # Check if the point is within the polygon
    result = point.within(gdf_polygon)

    return result



In [65]:
# Used to convert street names to E911 format
number_to_words = {
      '1st': 'first',
      '2nd': 'second',
      '3rd': 'third',
      '4th': 'fourth',
      '5th': 'fifth',
      '6th': 'sixth',
      '7th': 'seventh',
      '8th': 'eighth',
      '9th': 'ninth',
      '10th': 'tenth',
      '11th': 'eleventh',
      '12th': 'twelfth',
      '13th': 'thirteenth',
      '14th': 'fourteenth',
      '15th': 'fifteenth',
  }

In [66]:
# Get block coordinates
def get_block_coordinates(components, midpoint):
    block, street = components
    block = int(block)
    stsuffix_to_abbr = {
        'avenue': 'av',
        'ave': 'av',
        'street': 'st',
        'terrace': 'terr',
        'ter': 'terr',
        'boulevard': 'blvd',
        'drive': 'dr',
        'road': 'rd',
        'way': 'wy',
        'lane': 'ln',
        'court': 'ct',
        'place': 'pl',
        'parkway': 'pkwy',
        'square': 'sq',
        'walk': 'wk',
        'plaza': 'plz',
        'circle': 'cir'}

    # Standardize street name to match E911 format
    # Need to account for cases where streets are written slightly differently (ex. 'street' vs 'st', 'ave' vs 'av)
    street = street.lower().strip()
    # Remove periods from street names
    street = street.replace('.', '')
    street = street + ' '
    # Reformat street suffixes
    for k, v in stsuffix_to_abbr.items():
        street = street.replace(f" {k} ", f" {v} ")
    # Spell out numerical street names
    for k, v in number_to_words.items():
        street = street.replace(k, v)
    # Remove trailing space
    street = street.rstrip()

    # Find E911 sites with matching street names / street name aliases
    df = gdf[(gdf['St_Full'] == street) | (gdf['St_Alias1'] == street) | (gdf['St_Alias2'] == street) | (gdf['St_Alias3'] == street) | (gdf['St_Alias4'] == street) | (gdf['St_Alias5'] == street)]

    # Try a more flexible strategy if no matches are returned
    if len(df) == 0:
        # Handle case where crime log location contains extra words/characters
        def filter_fn(row):
            if row['St_Full']:
                # Remove apostrophes to minimize inconsistencies (e.g. "o'connell" vs "oconnell")
                if row['St_Full'].replace('\'', '') in street.replace('\'', ''):
                    return True
            return False
        df_temp = gdf[gdf.apply(filter_fn, axis=1)]
        if len(df_temp) > 0:
            df = df_temp
        else:
            # Handle case where street suffix is missing
            def filter_fn2(row):
                if row['St_Name']:
                    if row['St_Name'].replace('\'', '') in street.replace('\'', ''):
                        return True
                return False
            df_temp = gdf[gdf.apply(filter_fn2, axis=1)]
            df_temp = df_temp[(df_temp['Add_Number'] >= block) & (df_temp['Add_Number'] < block + 100) & (df_temp['Add_Number'] != 0)]
            # verify that there is only one street suffix for the given street name after filtering
            if len(df_temp['St_PosType'].unique()) == 1:
                df = df_temp
        
    # Filter E911 sites by block number
    # Exclude E911 sites where 'Add_Number'==0; these correspond to E911 sites without address numbers
    df = df[(df['Add_Number'] >= block) & (df['Add_Number'] < block + 100) & (df['Add_Number'] != 0)]
    df = df.sort_values(by='Add_Number', ascending=True)

    if len(df) > 0:
        # Midpoint method
        if midpoint:
            latitude = (df['Latitude'].iloc[0] + df['Latitude'].iloc[-1])/2
            longitude = (df['Longitude'].iloc[0] + df['Longitude'].iloc[-1])/2

        # Middle house method
        else:
            if len(df) % 2 == 1:
                middle_row = df.iloc[df.shape[0] // 2]
                latitude = middle_row['Latitude']
                longitude = middle_row['Longitude']
            else:
                middle_two_rows = df.iloc[df.shape[0] // 2 - 1 : df.shape[0] // 2 + 1]
                latitude = middle_two_rows['Latitude'].mean()
                longitude = middle_two_rows['Longitude'].mean()
        return latitude, longitude
    return 0, 0
        

In [67]:
# Used to convert from RI State Plane system to WGS 84
reproject = Transformer.from_crs(3438, 4326, always_xy=True)

def get_intersection_coords(address):
    """ Geocode intersections with RIDOT """
    base_url_ad='https://risegis.ri.gov/gpserver/rest/services/E911_StreetRange_Locator/GeocodeServer/findAddressCandidates?'
    address=address.replace(' & ',' and ')
    city='Providence'
    try:
        add_url=f'Street={address}&City={city}'
        data_url = f'{base_url_ad}{add_url}&maxLocations=5&matchOutOfRange=true&WriteXYCoordFields=false&f=pjson'
        response=requests.get(data_url)
        add_data=response.json()['candidates'] # Collapse the dictionary by one level
        if len(add_data)==0:
            pass
        elif len(add_data)==1:
            longitude=add_data[0]['location']['x']
            latitude=add_data[0]['location']['y'] 
            longitude, latitude = reproject.transform(longitude, latitude)
            return latitude, longitude
        else:        
            all_scores=[]
            for m in add_data:
                all_scores.append(m['score'])
            maxs=max(all_scores) # Find highest score
            maxs_idx=all_scores.index(maxs) # And its index (takes 1st highest value if several are equal)
            # Get data for highest match and store
            longitude=add_data[maxs_idx]['location']['x']
            latitude=add_data[maxs_idx]['location']['y']
            longitude, latitude = reproject.transform(longitude, latitude)
            return latitude, longitude
    except Exception as e:
            print(str(e))
    return 0, 0

In [68]:
# Load landmark coordinates
df_landmark = pd.read_excel(os.path.join('.', 'inputs', 'landmarks.xlsx'))
df_landmark['aliases'] = df_landmark['aliases'].astype(str).apply(lambda x: x.split(', '))
df_landmark['aliases'] = df_landmark.apply(lambda x: x['aliases'] + [x['location']], axis=1)
df_landmark['aliases'] = df_landmark['aliases'].apply(lambda x: [alias.lower() for alias in x])
df_by_alias = df_landmark.explode('aliases', ignore_index=True)
df_by_alias['aliases'] = df_by_alias['aliases'].drop_duplicates()

def is_street(address):
    address = address.lower().strip()
    street_indicators = [' street', ' st',  ' st.', ' ave', ' av', ' avenue', ' blvd', ' rd', ' way', ' dr']
    for indicator in street_indicators:
        if address.endswith(indicator):
            return True
    return False

def get_landmark_coords(address):
    try:
        row = df_by_alias[df_by_alias['aliases'] == address.lower().strip()]
        lat = float(row['latitude'])
        long = float(row['longitude'])
        return lat, long
    except:
        if not is_street(address):
            # Print the unrecognized landmark
            print(address)
        return 0, 0

# Test
get_landmark_coords('kennedy')

(41.82497, -71.41162)

In [69]:
# Providence boundaries (with 0.005 degree buffer)
min_lat = 41.7673
max_lat = 41.8668
min_long = -71.4774
max_long = -71.3719

##### Not needed -- in_providence() is used instead #####
def within_bounds(latitude, longitude):
        """Returns True if the coordinates are within the Providence boundaries"""
        if latitude > min_lat and latitude < max_lat and longitude > min_long and longitude < max_long:
            return True
        return False

In [70]:
def get_coordinates(address, midpoint):
    """Returns a tuple of numerical (Latitude, Longitude) coordinates in WGS 84
    Parameter:
              address (string): the 'location' value from the case logs DataFrame
              midpoint (boolean): indicates whether the midpoint method should be used in lieu of
              the middle house method to calculate block centers
              """
    # Get address category and necessary components for geocoding
    category, components = categorize_address(address)

    # Geocode block locations
    if category == 0:
        latitude, longitude = get_block_coordinates(components, midpoint)
        if in_providence(latitude, longitude):
            return latitude, longitude, 'E911'

    # Geocode intersection locations
    elif category == 1:
        latitude, longitude = get_intersection_coords(address)
        if in_providence(latitude, longitude):
            return latitude, longitude, 'RIDOT'

    # Geocode landmarks
    else:
      latitude, longitude = get_landmark_coords(address)
      if in_providence(latitude, longitude):
         return latitude, longitude, 'Landmark File'
              
    # Return None if no coordinates are found
    return None, None, None

In [71]:
final_df = results_df.copy()
# Set to False to use middle house method for calculating block centers
use_midpoint = True

print('Unrecognized landmarks (if any):')

# Apply our geocoding function to add latitude and longitude columns
final_df[['latitude', 'longitude', 'source']] = final_df['location'].apply(lambda x: pd.Series(get_coordinates(x, use_midpoint), dtype=object))
# Store records that could not be geocoded
no_matches_df = final_df[pd.isnull(final_df['latitude'])]
print(len(no_matches_df))
# Store records that were successfully geocoded
final_df = final_df[pd.notna(final_df['latitude'])]

Unrecognized landmarks (if any):
445


# TODO: Update the landmark file #
Update 'landmarks.xlsx' with the appropriate coordinates. Coordinates can be found 
[here](https://www.openstreetmap.org/search?query=#map=12/41.8173/-71.4231)
by searching a location name, right clicking on the map, and selecting "Show address".
Invalid landmarks can be added to the spreadsheet with the coordinates (0, 0) to suppress future 
print statements.


In [72]:
print(len(final_df), "records successfully geocoded")
print(len(no_matches_df), "records could not be geocoded")

0 records successfully geocoded
445 records could not be geocoded


### Assign unique IDs

In [73]:
if geocode_all:
    final_df_copy = final_df
# Concatenate past and present DFs
else:
    final_df_copy = pd.concat((past_df, final_df), axis=0, ignore_index=True)

# Assign unique IDs to each offense
casenum_counts = {casenum: 0 for casenum in final_df_copy['casenumber']}

def generate_unique_id(row):
    casenum = row['casenumber']
    casenum_counts[casenum] += 1
    # Add 3 digits to each original case number
    unique_id = casenum + '-' + str(casenum_counts[casenum]).zfill(3)
    return unique_id
final_df_copy['unique_id'] = final_df_copy.apply(generate_unique_id, axis=1)

# Do the same for unmatched locations
if exclude_past_no_match and not geocode_all:
    # Add new list to previous list of no matches
    no_matches_df_new = pd.concat((past_no_matches, no_matches_df))
else:
    # If we tried to geocode the previous list of no matches, we don't need to add it again
    no_matches_df_new = no_matches_df

# Do the same for no_matches_df_new
casenum_counts = {casenum: 0 for casenum in no_matches_df_new['casenumber']}

def generate_unique_id(row):
    casenum = row['casenumber']
    casenum_counts[casenum] += 1
    unique_id = casenum + '-' + str(casenum_counts[casenum]).zfill(3)
    return unique_id
no_matches_df_new['unique_id'] = no_matches_df_new.apply(generate_unique_id, axis=1)
no_matches_df_new.head()


,casenumber,location,reported_date,month,year,offense_desc,statute_code,statute_desc,counts,reporting_officer,unique_id,violent_cat,property_cat,latitude,longitude,source
30,2024-00001955,FINANCE WAY & MEMORIAL BLVD,2024-01-08T17:09:00.000,1,2024,Request for Assistance,Not Used,No Violations,0,DAnderson,2024-00001955-001,NaN,NaN,None,None,None
33,2024-00500049,S CRIMEA,2024-01-08T15:06:59.000,1,2024,Larceny from Motor Vehicle,11-41-1,LARCENY/U $1500 - FROM MV,1,NField,2024-00500049-001,NaN,NaN,None,None,None
34,2024-00001924,F C GREENE MEMORIAL BLVD,2024-01-08T14:57:00.000,1,2024,"Larceny, Other",11-41-7,LARCENY FROM THE PERSON - ALL OTH LARCENY,1,Central Station,2024-00001924-001,NaN,NaN,None,None,None
42,2024-00001868,BROAD ST,2024-01-08T09:31:00.000,1,2024,"Larceny, Other",11-41-1,LARCENY/U $1500 - ALL OTH LARCENY,1,Central Station,2024-00001868-001,NaN,NaN,None,None,None
46,2024-00001837,600 Block PLEASANT VALLEY PKY,2024-01-08T07:23:00.000,1,2024,Property Damage,Not Used,No violations,0,KBass,2024-00001837-001,NaN,NaN,None,None,None


# TODO: Update the crime categorization file
If any unrecognized offense descriptions are printed out, open 'crime_cats.xlsx' and
manually enter the new offense descriptions along with their appropriate crime
categorizations. Then, rerun this cell and all following cells; 
no unrecognized offense descriptions should be printed.

In [74]:
file_path = os.path.join('inputs' ,'crime_cats.xlsx')
df = pd.read_excel(file_path)

# Create dictionaries
vc = dict(zip(df['offense_desc'], df['violent_cat']))
pc = dict(zip(df['offense_desc'], df['property_cat']))

def get_categories(offense_desc):
    if offense_desc in vc:
        return vc[offense_desc], pc[offense_desc]
    else:
        print('Unrecognized offense description:', offense_desc)
        return None, None
categorized = final_df_copy.copy()
categorized[['violent_cat', 'property_cat']] = categorized['offense_desc'].apply(lambda x: pd.Series(get_categories(x)))

# Do the same for no_matches_df_new
no_matches_df_new[['violent_cat', 'property_cat']] = no_matches_df_new['offense_desc'].apply(lambda x: pd.Series(get_categories(x)))
no_matches_df_new.head()

,casenumber,location,reported_date,month,year,offense_desc,statute_code,statute_desc,counts,reporting_officer,unique_id,violent_cat,property_cat,latitude,longitude,source
30,2024-00001955,FINANCE WAY & MEMORIAL BLVD,2024-01-08T17:09:00.000,1,2024,Request for Assistance,Not Used,No Violations,0,DAnderson,2024-00001955-001,None,None,None,None,None
33,2024-00500049,S CRIMEA,2024-01-08T15:06:59.000,1,2024,Larceny from Motor Vehicle,11-41-1,LARCENY/U $1500 - FROM MV,1,NField,2024-00500049-001,None,Larceny-theft,None,None,None
34,2024-00001924,F C GREENE MEMORIAL BLVD,2024-01-08T14:57:00.000,1,2024,"Larceny, Other",11-41-7,LARCENY FROM THE PERSON - ALL OTH LARCENY,1,Central Station,2024-00001924-001,None,Larceny-theft,None,None,None
42,2024-00001868,BROAD ST,2024-01-08T09:31:00.000,1,2024,"Larceny, Other",11-41-1,LARCENY/U $1500 - ALL OTH LARCENY,1,Central Station,2024-00001868-001,None,Larceny-theft,None,None,None
46,2024-00001837,600 Block PLEASANT VALLEY PKY,2024-01-08T07:23:00.000,1,2024,Property Damage,Not Used,No violations,0,KBass,2024-00001837-001,None,None,None,None,None


### Save the output files

In [75]:
no_matches_df_new = no_matches_df_new.drop(columns=['latitude', 'longitude', 'source'])

In [79]:
final_df = categorized
final_df['year'] = final_df['year'].astype(int)
no_matches_df_new['year'] = no_matches_df_new['year'].astype(int)

reproject = Transformer.from_crs(4326, 3438, always_xy=True)

overwrite_files = True
if overwrite_files:
    # Shapefile should be in RI State Plane system
    long_3438, lat_3438 = final_df[['latitude', 'longitude']].apply(lambda x: reproject.transform(x[1], x[0]), axis=1).apply(pd.Series).values.T
    shp_df = gpd.GeoDataFrame(final_df, geometry=gpd.points_from_xy(long_3438, lat_3438), crs = 'EPSG:3438')

    # Save the GeoDataFrame to a Shapefile
    shp_df.to_file(os.path.join("..", "outputs", "all", "geocoded_results.shp"), index=False)

    # Save separate Shapefiles by year
    for year in final_df['year'].unique():
        try: 
            # Make a new directory for the year if it doesn't already exist
            os.mkdir(os.path.join("..", "outputs", str(year)))
        except:
            pass
        year_df = final_df[final_df['year'] == year]
        long_3438, lat_3438 = year_df[['latitude', 'longitude']].apply(lambda x: reproject.transform(x[1], x[0]), axis=1).apply(pd.Series).values.T
        shp_df = gpd.GeoDataFrame(year_df, geometry=gpd.points_from_xy(long_3438, lat_3438), crs = 'EPSG:3438')
        shp_df.to_file(os.path.join("..", "outputs", str(year), f"geocoded_results_{year}.shp"), index=False)

    # Save the DataFrame to a CSV
    final_df = final_df.drop(columns=['geometry'])
    final_df.to_csv(os.path.join("..", "outputs", "all", "geocoded_results.csv"), index=False)

    # Save the locations we have failed to geocode
    no_matches_df_new.to_csv(os.path.join("..", "outputs", "all", "no_location_matches.csv"), index=False)

    # Save separate CSV files by year
    for year in final_df['year'].unique():
        year_df = final_df[final_df['year'] == year]
        year_df.to_csv(os.path.join("..", "outputs", str(year), f"geocoded_results_{year}.csv"), index=False)
        no_matches_year_df = no_matches_df_new[no_matches_df_new['year'] == year]
        no_matches_year_df.to_csv(os.path.join("..", "outputs", str(year), f"no_location_matches_{year}.csv"), index=False)
